Created: 2020.07.03

Modified: 2020.08.10

#### Convertion 3D isotropic images to set of 2D png slices based on paths in a df: train_val_mk_3d.csv

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import glob
import pandas as pd
import numpy as np
import imageio as io
from pathlib import Path
from fastai.core import parallel

In [3]:
import sys
sys.path.append('functions')

%aimport functions00, functions01
from functions00 import *
from functions01 import *

import functions23 as fun

In [4]:
mk_get_host_info()

**********************************************************************************************************************************
Settings:
	HOST:  mmiv-ml-titan
	PATH_ROOT_DATA:  /data-10tb/shared/skull/train-3d-iso
	PATH_GIT_HUB:  /data-10tb/marek/github_codes/skull-stripping-1/fastai
	PATH_2D: /data-10tb/shared/skull

3D DFs paths (_mk_3D):
	IXI_TEST_3D: /data-10tb/marek/github_codes/skull-stripping-1/fastai/2.2_train_valid_test_sets/ixi_test_mk_3d.csv
	TEST_3D : /data-10tb/marek/github_codes/skull-stripping-1/fastai/2.2_train_valid_test_sets/test_mk_3d.csv
	TRAIN_VAL_3D: /data-10tb/marek/github_codes/skull-stripping-1/fastai/2.2_train_valid_test_sets/train_val_mk_3d.csv
	NFBS_TEST_3D: /data-10tb/marek/github_codes/skull-stripping-1/fastai/2.2_train_valid_test_sets/nfbs_test_mk_3d.csv

Error files (pickle):
	ERROR_FILES: /data-10tb/marek/github_codes/skull-stripping-1/fastai/2.2_train_valid_test_sets/error_files
**********************************************************************

In [5]:
# pd.read_csv(...., index_col=0)
df = pd.read_csv(TRAIN_VAL_3D, index_col=None)
print(df.shape)

(2791, 16)


In [6]:
df.head(2)

,t1_pth_dgx,bias_pth_dgx,mask_nn_pth_dgx,mask_lin_pth_dgx,t1_pth_titan,bias_pth_titan,mask_nn_pth_titan,mask_lin_pth_titan,age_at_scan,gender,subject_id,img_id,source,rounded_age,age_group,usage
0,/data-external/sathiesh/br...,/data-external/sathiesh/br...,/data-external/sathiesh/br...,/data-external/sathiesh/br...,/data-10tb/shared/skull/tr...,/data-10tb/shared/skull/tr...,/data-10tb/shared/skull/tr...,/data-10tb/shared/skull/tr...,74.3,M,011_S_0002,35475.0,ADNI,74,74,train
1,/data-external/sathiesh/br...,/data-external/sathiesh/br...,/data-external/sathiesh/br...,/data-external/sathiesh/br...,/data-10tb/shared/skull/tr...,/data-10tb/shared/skull/tr...,/data-10tb/shared/skull/tr...,/data-10tb/shared/skull/tr...,78.5,F,022_S_0014,59375.0,ADNI,78,79,train


### Remove bad images from a df on ERROR_FILES

In [7]:
mk_remove_error_flies_from_df(df, mk_get_error_file_list() )

None of the files from the error_list is contained in a df!


## TRAINING

In [8]:
df_train = df[df.usage=='train']
print(df_train.shape)

(2132, 16)


#### Prepare a list of all 3D images to split into 2d images

In [9]:
im3d_train = []
if 'dgx'.upper() in HOST:
    im3d_train.extend(df_train['t1_pth_dgx'])
    im3d_train.extend(df_train['bias_pth_dgx'])
    im3d_train.extend(df_train['mask_nn_pth_dgx'])
if 'titan' in HOST:
    im3d_train.extend(df_train['t1_pth_titan'])
    im3d_train.extend(df_train['bias_pth_titan'])
    im3d_train.extend(df_train['mask_nn_pth_titan'])

len(im3d_train)

6396

#### Check in all files exists

In [10]:
if 'titan' in HOST:
    #df_files_exists(df, ['t1_pth_titan', 'bias_pth_titan', 'mask_nn_pth_titan', 'mask_lin_pth_titan'])
    mk_df_files_exists(df, ['t1_pth_titan', 'bias_pth_titan', 'mask_nn_pth_titan'])
    

Looing for NOT EXISTING files in "df.t1_pth_titan"
	All files are OK

Looing for NOT EXISTING files in "df.bias_pth_titan"
	All files are OK

Looing for NOT EXISTING files in "df.mask_nn_pth_titan"
	All files are OK



In [11]:
if 'dgx'.upper() in HOST:
    mk_df_files_exists(df, ['t1_pth_dgx', 'bias_pth_dgx', 'mask_nn_pth_dgx', 'mask_lin_pth_dgx'])

### Prepare to splitting

In [12]:
im3d_train_folder = list(map(lambda e: (e, 'train'), im3d_train))

#### Axial

In [13]:
%%time
parallel(fun.axial_3d_2_2d, im3d_train_folder, max_workers=32)

CPU times: user 15 s, sys: 2.45 s, total: 17.5 s
Wall time: 19min 44s


#### Coronal

In [14]:
%%time
parallel(fun.coronal_3d_2_2d, im3d_train_folder, max_workers=32)

CPU times: user 14.8 s, sys: 2.45 s, total: 17.2 s
Wall time: 16min 47s


#### Sagittal

In [15]:
%%time
parallel(fun.sagittal_3d_2_2d, im3d_train_folder, max_workers=32)

CPU times: user 16.9 s, sys: 3.06 s, total: 20 s
Wall time: 20min 44s


## VALIDATION

In [16]:
df_val = df[df.usage=='val']
df_val.shape

(659, 16)

#### Prepare a list of all 3D images to split into 2d images

In [17]:
im3d_val = []
if 'dgx'.upper() in HOST:
    im3d_val.extend(df_val['t1_pth_dgx'])
    im3d_val.extend(df_val['bias_pth_dgx'])
    im3d_val.extend(df_val['mask_nn_pth_dgx'])
if 'titan' in HOST:
    im3d_val.extend(df_val['t1_pth_titan'])
    im3d_val.extend(df_val['bias_pth_titan'])
    im3d_val.extend(df_val['mask_nn_pth_titan'])

len(im3d_val)

1977

#### Check in all files exists

In [18]:
if 'titan' in HOST:
    #df_files_exists(df, ['t1_pth_titan', 'bias_pth_titan', 'mask_nn_pth_titan', 'mask_lin_pth_titan'])
    mk_df_files_exists(df, ['t1_pth_titan', 'bias_pth_titan', 'mask_nn_pth_titan'])

Looing for NOT EXISTING files in "df.t1_pth_titan"
	All files are OK

Looing for NOT EXISTING files in "df.bias_pth_titan"
	All files are OK

Looing for NOT EXISTING files in "df.mask_nn_pth_titan"
	All files are OK



In [19]:
if 'dgx'.upper() in HOST:
    mk_df_files_exists(df, ['t1_pth_dgx', 'bias_pth_dgx', 'mask_nn_pth_dgx', 'mask_lin_pth_dgx'])

### Prepare to splitting

In [20]:
im3d_val_folder = list(map(lambda e: (e, 'valid'), im3d_val))

#### Axial

In [21]:
%%time
parallel(fun.axial_3d_2_2d, im3d_val_folder, max_workers=32)

CPU times: user 4.38 s, sys: 892 ms, total: 5.27 s
Wall time: 9min 3s


#### Coronal

In [22]:
%%time
parallel(fun.coronal_3d_2_2d, im3d_val_folder, max_workers=32)

CPU times: user 4.44 s, sys: 978 ms, total: 5.41 s
Wall time: 5min 26s


#### Sagittal

In [23]:
%%time
parallel(fun.sagittal_3d_2_2d, im3d_val_folder, max_workers=32)

CPU times: user 4.66 s, sys: 1.05 s, total: 5.72 s
Wall time: 3min 52s


## Previous version

##### Replace values bigger than 1 to 1 in ALL MASK images

In [24]:
# pth = Path('/data-10tb/shared/skull/')
# pngs = sorted(glob.glob('/data-10tb/shared/skull/*/*/*/*/*_mask_iso*.png'))
# len(pngs)

In [25]:
# l = len(pngs)
# for i, fname in enumerate(pngs):
#     io.imsave(fname, np.where(io.imread(fname)>=1, 1, 0).astype('uint8'))
#     print(f'{i}/{l}', end='\r')